In [90]:
# Dependencies
import requests
import pandas as pd
from census import Census
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
from scipy.stats import linregress
from config import api_key

In [91]:
## Pulling Median Income By County (CA)

years = [2018,2019,2021,2022] # no 2020 data available due to COVID -19
med_income_var = 'B19013_001E'
base_url = 'https://api.census.gov/data/'
geo = '&for=county:*&in=state:06'  # California = 06
dfs = []

for year in years: 
    url = f'{base_url}{year}/acs/acs1?get={med_income_var}{geo}&key={api_key}'
    
    data = requests.get(url).json()

    data = pd.DataFrame(data[1:])
   
    data["Year"] = year

    data = data.rename(columns={0: 'Median Income',1: 'State',2:'County'})
    dfs.append(data)

income_data_county = pd.concat(dfs, ignore_index=True)
income_data_county['State'] = pd.to_numeric(income_data_county['State'])
income_data_county['County'] = pd.to_numeric(income_data_county['County'])
income_data_county["Median Income"] = income_data_county["Median Income"].astype('int')
income_data_county

,Median Income,State,County,Year
0,48554,6,33,2018
1,57745,6,47,2018
2,101618,6,13,2018
3,60321,6,99,2018
4,77472,6,83,2018
...,...,...,...,...
160,58884,6,103,2022
161,64722,6,107,2022
162,102569,6,111,2022
163,83875,6,113,2022


In [92]:
## Pulling Median Income By State (CA)

years = [2018,2019,2021,2022] # no 2020 data available due to COVID -19
med_income_var = 'B19013_001E'
base_url = 'https://api.census.gov/data/'
geo = '&for=state:06'  # California = 06
dfs = []

for year in years: 
    url = f'{base_url}{year}/acs/acs1?get={med_income_var}{geo}&key={api_key}'

    data = requests.get(url).json()

    data = pd.DataFrame(data[1:])
   
    data["Year"] = year

    data = data.rename(columns={0: 'Median Income',1: 'State',2:'County'})
    dfs.append(data)

income_data_state = pd.concat(dfs, ignore_index=True)
income_data_state['State'] = pd.to_numeric(income_data_state['State'])
income_data_state["Median Income"] = income_data_state["Median Income"].astype('int')
income_data_state

,Median Income,State,Year
0,75277,6,2018
1,80440,6,2019
2,84907,6,2021
3,91551,6,2022


In [93]:
#Pulling Geocodes for State(CA)
geocodes_csv = "../Madison/Resources/geocodes.csv"
geo_state = pd.read_csv(geocodes_csv)
geo_state = geo_state.rename(columns={"Area Name (including legal/statistical area description)":"Area","State Code (FIPS)":"State"})
geo_state = geo_state[(geo_state['State']== 6) & (geo_state["County Code (FIPS)"]==0) *(geo_state["Place Code (FIPS)"] == 0)]
geo_state = geo_state[["State","Area"]]
geo_state

,State,Area
1393,6,California


In [94]:
#Pulling Geocodes for County(CA)
geocodes_csv = "../Madison/Resources/geocodes.csv"
geo_county = pd.read_csv(geocodes_csv)
geo_county = geo_county.rename(columns={"Area Name (including legal/statistical area description)":"Area","State Code (FIPS)":"State","County Code (FIPS)":"County"})
geo_county = geo_county[['State','County','Area']]
geo_county = geo_county[(geo_county['State']== 6) & (geo_county["County"] > 0)]
geo_county.head()

,State,County,Area
1394,6,1,Alameda County
1395,6,3,Alpine County
1396,6,5,Amador County
1397,6,7,Butte County
1398,6,9,Calaveras County


In [95]:
#Created CSV for Regions in case we want to do Socal/NorCal Analysis
state_split_csv = "../Madison/Resources/California - Counties.csv"
state_split = pd.read_csv(state_split_csv)
state_split.head()

,County,Region
0,Alameda County,NorCal
1,Alpine County,NorCal
2,Amador County,NorCal
3,Butte County,NorCal
4,Calaveras County,NorCal


In [96]:
#Merging DF's for County Names
data_county = pd.merge(income_data_county,geo_county,on=['State','County'],how='left')
data_county.drop(columns=["County"], inplace=True)
data_county.rename(columns={'Area': 'County'},inplace=True)
data_county['State'] = 'California'
data_county  = pd.merge(data_county,state_split, on= "County",how='left')
data_county = data_county[['Year', 'State','Region','County','Median Income']]
data_county.head()

,Year,State,Region,County,Median Income
0,2018,California,NorCal,Lake County,48554
1,2018,California,NorCal,Merced County,57745
2,2018,California,NorCal,Contra Costa County,101618
3,2018,California,NorCal,Stanislaus County,60321
4,2018,California,SoCal,Santa Barbara County,77472


In [97]:
#Merging DF's for State Name (Can easily rename column to 'California' but merging will ensure accuracy of pull)
data_state = pd.merge(income_data_state,geo_state,on=['State'],how='left')
data_state.drop(columns=["State"],inplace=True)
data_state.rename(columns={"Area":"State"},inplace=True)
data_state = data_state[["Year","State","Median Income"]]
data_state

,Year,State,Median Income
0,2018,California,75277
1,2019,California,80440
2,2021,California,84907
3,2022,California,91551


In [98]:
#Working with Median Housing Price Data - Cleaning up and getting Mean by year

pd.set_option('display.float_format', '{:.2f}'.format)

csv = "../Madison/Resources/Median(New) - MedianPricesofExistingDetachedHomesHistoricalData.csv"

median_data = pd.read_csv(csv)

median_data = median_data.drop('Unnamed: 54',axis=1)
median_data = median_data.astype('str')

def clean_currency(value):
    try:
        return int(str(value).replace(',', '').replace('$', ''))
    except ValueError:
        # Return np.nan for non-convertible values
        return np.nan

columns_to_clean = ['CA', 'Alameda', 'Amador', 'Butte', 'Calaveras',
       'Contra-Costa', 'Del Norte', 'El Dorado', 'Fresno', 'Glenn', 'Humboldt',
       'Kern', 'Kings', 'Lake', 'Lassen', 'Los Angeles', 'Madera', 'Marin',
       'Mariposa', 'Mendocino', 'Merced', 'Mono', 'Monterey', 'Napa', 'Nevada',
       'Orange', 'Placer', 'Plumas', 'Riverside', 'Sacramento', 'San Benito',
       'San Bernardino', 'San Diego', 'San Francisco', 'San Joaquin',
       'San Luis Obispo', 'San Mateo', 'Santa Barbara', 'Santa Clara',
       'Santa Cruz', 'Shasta', 'Siskiyou', 'Solano', 'Sonoma', 'Stanislaus',
       'Sutter', 'Tehama', 'Trinity', 'Tulare', 'Tuolumne', 'Ventura', 'Yolo',
       'Yuba', 'Condo', 'LA Metro', 'Central Coast', 'Central Valley',
       'Far North', 'Inland Empire', 'S.F. Bay Area', 'SoCal']
median_data[columns_to_clean] = median_data[columns_to_clean].applymap(clean_currency)
median_data["Mon-Yr"] = pd.to_datetime(median_data["Mon-Yr"],format='%b-%y')
median_data = median_data[median_data["Mon-Yr"] >= "2018-01-01"]

median_data["Year"] = median_data["Mon-Yr"].dt.year
median_data = median_data[['Year', 'CA', 'Alameda', 'Amador', 'Butte', 'Calaveras',
       'Contra-Costa', 'Del Norte', 'El Dorado', 'Fresno', 'Glenn', 'Humboldt',
       'Kern', 'Kings', 'Lake', 'Lassen', 'Los Angeles', 'Madera', 'Marin',
       'Mariposa', 'Mendocino', 'Merced', 'Mono', 'Monterey', 'Napa', 'Nevada',
       'Orange', 'Placer', 'Plumas', 'Riverside', 'Sacramento', 'San Benito',
       'San Bernardino', 'San Diego', 'San Francisco', 'San Joaquin',
       'San Luis Obispo', 'San Mateo', 'Santa Barbara', 'Santa Clara',
       'Santa Cruz', 'Shasta', 'Siskiyou', 'Solano', 'Sonoma', 'Stanislaus',
       'Sutter', 'Tehama', 'Trinity', 'Tulare', 'Tuolumne', 'Ventura', 'Yolo',
       'Yuba']]
median_data.reset_index().drop(columns=["index"])
median_grp = median_data.groupby("Year")

median_data = median_grp.mean().reset_index()
median_data

,Year,CA,Alameda,Amador,Butte,Calaveras,Contra-Costa,Del Norte,El Dorado,Fresno,...,Sonoma,Stanislaus,Sutter,Tehama,Trinity,Tulare,Tuolumne,Ventura,Yolo,Yuba
0,2018,571057.50,926358.33,327476.67,320112.50,321708.33,649169.17,230191.67,491687.42,268630.83,...,667375.00,312658.33,293766.67,219012.50,NaN,232981.67,295350.83,654005.83,439729.17,278813.33
1,2019,591865.83,911500.00,324687.50,361633.33,339101.67,661736.67,261485.83,497453.33,281920.83,...,655243.33,326854.17,310212.50,255875.00,258329.17,243641.67,301125.00,651220.83,452575.83,294937.50
2,2020,650156.67,1001271.67,346366.67,384780.83,366409.17,737000.00,290100.00,529164.17,309070.83,...,695219.17,359583.33,339108.33,276604.17,296495.83,267141.67,317875.00,714145.83,485400.83,333161.67
3,2021,785640.83,1242958.33,421805.83,434448.33,459687.50,902541.67,365845.83,646720.00,367208.33,...,767266.67,425541.67,399816.67,327787.50,327479.17,318210.00,380051.67,819937.50,575560.83,401433.33
4,2022,821204.17,1306546.67,418425.00,446898.33,473070.83,903200.00,375087.50,671083.33,408416.67,...,826887.50,455695.00,431525.00,327394.17,303500.00,357455.83,413500.83,886541.67,623743.33,428329.17
5,2023,811128.89,1221111.11,445243.33,427863.33,480388.89,853632.22,352388.89,653413.33,409376.67,...,831598.89,450738.89,414500.00,321958.89,316343.75,361255.56,407450.00,889444.44,601711.11,435237.78


In [99]:
#melt(take columns and make them rows) Housing Median Price DF into two DFS: One For CA and one by County.  
# Also merging with Median Income DF's 

melted_df = pd.melt(median_data, id_vars=['Year'], var_name='Region', value_name='Median Housing Price')
melted_df["Region"] = melted_df["Region"] + " County"
melted_df.loc[melted_df['Region'] == 'CA County', 'Region'] = 'California'

#create county DF(Housing Prices)
housing_county = melted_df[melted_df['Region']!= 'California'].copy()
housing_county.rename(columns={"Region":"County"},inplace=True)

#create state DF (Housing Prices)
housing_state = melted_df[(melted_df['Region']== 'California') & (melted_df['Year'] != 2020) & (melted_df["Year"] != 2023)].copy()

housing_state = housing_state.reset_index().drop(columns=['index'])
housing_state.rename(columns={"Region":"State"},inplace=True)
state_df = pd.merge(data_state,housing_state,on=["Year","State"])
county_df = pd.merge(data_county,housing_county,on=["Year","County","County"])

county_df.head()

,Year,State,Region,County,Median Income,Median Housing Price
0,2018,California,NorCal,Lake County,48554,269408.33
1,2018,California,NorCal,Merced County,57745,268086.67
2,2018,California,NorCal,Stanislaus County,60321,312658.33
3,2018,California,SoCal,Santa Barbara County,77472,662729.17
4,2018,California,NorCal,Sacramento County,69767,365454.17


In [100]:
#Store counties as a list for x-axis

median_price_counties=['Alameda', 'Amador', 'Butte', 'Calaveras',
       'Contra-Costa', 'Del Norte', 'El Dorado', 'Fresno', 'Glenn', 'Humboldt',
       'Kern', 'Kings', 'Lake', 'Lassen', 'Los Angeles', 'Madera', 'Marin',
       'Mariposa', 'Mendocino', 'Merced', 'Mono', 'Monterey', 'Napa', 'Nevada',
       'Orange', 'Placer', 'Plumas', 'Riverside', 'Sacramento', 'San Benito',
       'San Bernardino', 'San Diego', 'San Francisco', 'San Joaquin',
       'San Luis Obispo', 'San Mateo', 'Santa Barbara', 'Santa Clara',
       'Santa Cruz', 'Shasta', 'Siskiyou', 'Solano', 'Sonoma', 'Stanislaus',
       'Sutter', 'Tehama', 'Trinity', 'Tulare', 'Tuolumne', 'Ventura', 'Yolo',
       'Yuba']
print(len(median_price_counties))


52


In [101]:
#Divide Counties into Northern/Southern CA Counties 
def remove_counties(c):
    return c.replace(" County", "")

#Northern CA DataFrame
norcal_counties=county_df.loc[county_df['Region']=='NorCal', 'County'].map(remove_counties).tolist()
norcal_df2=county_df.loc[county_df['Region']=='NorCal']
# norcal_df=median_data[['Year',*norcal_counties]].copy()
print(norcal_counties)


#Southern CA DataFrame 
socal_counties=county_df.loc[county_df['Region']=='SoCal', 'County'].map(remove_counties).tolist()
socal_df2=county_df.loc[county_df['Region']=='SoCal']
print(socal_counties)

['Lake', 'Merced', 'Stanislaus', 'Sacramento', 'El Dorado', 'Monterey', 'San Francisco', 'Tulare', 'Humboldt', 'Madera', 'Santa Cruz', 'Marin', 'Placer', 'Shasta', 'Solano', 'Butte', 'Kings', 'Fresno', 'Sutter', 'San Mateo', 'San Joaquin', 'Yuba', 'Sonoma', 'Alameda', 'Napa', 'Yolo', 'Nevada', 'Mendocino', 'Santa Clara', 'Lake', 'Yuba', 'Sonoma', 'Alameda', 'Napa', 'Yolo', 'Nevada', 'Mendocino', 'Santa Clara', 'Merced', 'Stanislaus', 'Tehama', 'Sacramento', 'El Dorado', 'Monterey', 'San Francisco', 'Tulare', 'Humboldt', 'Madera', 'Santa Cruz', 'Marin', 'Placer', 'Shasta', 'Solano', 'Butte', 'Kings', 'Fresno', 'Sutter', 'San Mateo', 'San Joaquin', 'Kings', 'Napa', 'San Joaquin', 'Shasta', 'Solano', 'Stanislaus', 'Madera', 'Marin', 'Mendocino', 'Merced', 'Monterey', 'Nevada', 'Placer', 'Sacramento', 'San Benito', 'San Francisco', 'San Mateo', 'Santa Clara', 'Santa Cruz', 'Sonoma', 'Sutter', 'Tehama', 'Tulare', 'Yolo', 'Yuba', 'Lake', 'Alameda', 'Butte', 'El Dorado', 'Fresno', 'Humboldt',

In [102]:
norcal_df2

,Year,State,Region,County,Median Income,Median Housing Price
0,2018,California,NorCal,Lake County,48554,269408.33
1,2018,California,NorCal,Merced County,57745,268086.67
2,2018,California,NorCal,Stanislaus County,60321,312658.33
4,2018,California,NorCal,Sacramento County,69767,365454.17
5,2018,California,NorCal,El Dorado County,82742,491687.42
...,...,...,...,...,...,...
151,2022,California,NorCal,Sutter County,65018,431525.00
152,2022,California,NorCal,Tehama County,58884,327394.17
153,2022,California,NorCal,Tulare County,64722,357455.83
155,2022,California,NorCal,Yolo County,83875,623743.33


In [103]:
socal_df2

,Year,State,Region,County,Median Income,Median Housing Price
3,2018,California,SoCal,Santa Barbara County,77472,662729.17
8,2018,California,SoCal,San Diego County,79079,631283.33
13,2018,California,SoCal,Riverside County,66964,401593.33
14,2018,California,SoCal,San Bernardino County,63857,289050.00
21,2018,California,SoCal,San Luis Obispo County,71148,616472.08
23,2018,California,SoCal,Kern County,51579,239865.00
24,2018,California,SoCal,Orange County,89759,815319.17
25,2018,California,SoCal,Ventura County,84566,654005.83
36,2018,California,SoCal,Los Angeles County,68093,572350.00
46,2019,California,SoCal,Los Angeles County,72797,591863.33


In [104]:
#Convert DataFrames to CSV Files

socal_df2.to_csv('SoCal_csv', index=False)
county_df.to_csv('CountyDF_csv', index=False)